In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/yelp-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import os
print(os.listdir("../input"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json
['yelp-dataset']


In [ ]:

users = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json') as fl:
    for i, line in enumerate(fl):
        users.append(json.loads(line))
dfusers = pd.DataFrame(users)
dfusers.head()

In [ ]:
elite = pd.DataFrame(dfusers[['user_id', 'elite']]) 

elite_up = (elite.assign(elite = elite['elite'].str.split(',')).explode('elite').reset_index(drop = True))
elite_up.head()

In [ ]:
friends = pd.DataFrame(dfusers[['user_id', 'friends']])
friends_up = (friends.assign(friends = friends['friends'].str.split(',')).explode('friends').reset_index(drop = True))
friends_up.head()

In [ ]:
tip = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json') as fl:
    for i, line in enumerate(fl):
        tip.append(json.loads(line))
dftip = pd.DataFrame(tip)
dftip.head()

In [ ]:
review = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        review.append(json.loads(line))
dfrev = pd.DataFrame(review)
dfrev.head()

In [ ]:
checkin = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json') as fl:
    for i, line in enumerate(fl):
        checkin.append(json.loads(line))
dfci = pd.DataFrame(checkin)
dfci.head()

In [ ]:
bus = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        bus.append(json.loads(line))
dfbus = pd.DataFrame(bus)
dfbus.head()

In [ ]:
categories = pd.DataFrame(dfbus[['business_id', 'categories']]) 
cat_up = (categories.assign(categories = categories['categories'].str.split(',')).explode('categories').reset_index(drop = True))
cat_up.head()

In [ ]:
hours = pd.DataFrame(dfbus[['business_id', 'hours']])
def hour(hours_dict):
    if hours_dict is None:
        return None
    hour_format = ', '.join([f"'{day}': '{hours}'" for day, hours in hours_dict.items()])
    return hour_format
hours['hours'] = hours['hours'].apply(hour)

In [ ]:
#splitting the dataframe by ',' and dividing multi-valued cell into a different row
hours_up = (hours.assign(hours = hours['hours'].str.split(',')).explode('hours').reset_index(drop = True))
hours_up.head()

In [ ]:
attributes = pd.DataFrame(dfbus[['attributes', 'business_id']])
attributes.head()
attributes['attributes'] = attributes['attributes'].apply(lambda x: [f"{key}: {value}" for key, value in x.items()] if x is not None else [])
attributes_up = attributes.explode('attributes').reset_index(drop=True)
attributes_up.head()

In [ ]:
from sqlalchemy import create_engine
!pip install PyMySql
import pymysql
import sqlalchemy

In [ ]:
username = 'A08' # Your team name
password = 'A08password'
host = 'testproject.mysql.database.azure.com'
database = 'A08'  # Your team name

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
    'ssl_ca': 'path_to/ca-cert.pem',
    'ssl_cert': 'path_to/client-cert.pem',
    'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args})

In [ ]:
dftip.to_sql(name = "tips", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
elite_up.to_sql(name = "elite_table", con = engine, if_exists='replace', index=False, chunksize=50000)

In [ ]:
cat_up.to_sql(name = "categories", con = engine, if_exists='replace', index=False, chunksize=50000)

In [ ]:
hours_up.to_sql(name = "hour", con = engine, if_exists='replace', index=False, chunksize=50000)

In [ ]:
attributes_up.to_sql(name = "Attribute_table", con = engine, if_exists='replace', index=True, chunksize=1000)

In [ ]:
friends_up.to_sql(name="yelp_friends",con = engine, if_exists='replace', index=False, chunksize=50000)

In [ ]:
dfrev.to_sql(name="yelp_rev",con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
dfbus.to_sql(name = "business", con = engine, if_exists='replace', index=True, dtype={'attributes': sqlalchemy.types.JSON, 'hours': sqlalchemy.types.JSON}, chunksize=50000)

In [ ]:
dfusers.to_sql(name = "users", con = engine, if_exists='replace', index=True, dtype={'elite': sqlalchemy.types.JSON, 'friends': sqlalchemy.types.JSON}, chunksize=50000)

In [ ]:
dfci.to_sql(name = "checkin", con = engine, if_exists='replace', index=True, dtype={'date': sqlalchemy.types.JSON}, chunksize=50000)